# Sign Language Digits
Kaggel data =https://www.kaggle.com/ardamavi/sign-language-digits-dataset

the scope of thos project to detect sign language digits (0-9) 

avilable data 

Image size: 100x100
Color space: RGB

#### Steps:

##### read data 
##### preprocessing 
##### convert color space to grey scale then thershold
##### save data as npy 
##### use keras (TF-based) to create model (CNN)
##### train
##### Predicte and Evaluate the Model


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.preprocessing import scale

import pandas as pd

%matplotlib notebook

In [4]:
path ='./Dataset'

image_list =[]
lable =[]

for r, d, f in os.walk(path):
    for folder in d:
        folder_path=os.path.join(r, folder)
        
        for r1, d1, f1 in os.walk(folder_path):
            for file in f1:
                if '.JPG' in file:
                    file_path =str(folder_path)+"/"+str(file)
                    sample = cv2.imread(file_path,0)
                    if sample.shape != (100,100):
                        sample = cv2.resize(sample,(100,100))
                    #thershold for the images
                    ret,thresh3 = cv2.threshold(sample,150,255,cv2.THRESH_TRUNC)
                    #rescaling the vakues to 0-1 range
                    X = scale( thresh3, axis=0, with_mean=True, with_std=True, copy=True )
                    image_list.append(X)
                    lable.append(int(folder))


/home/ahmed/anaconda3/envs/newenvt/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


### create numpy and save it to npy

In [5]:
print (len(image_list))
print (len(lable))
procesed_data = np.array(image_list)
lable_data = np.array(lable)

2062
2062


In [6]:
if not os.path.exists('npy_dataset/'):
            os.makedirs('npy_dataset/')
np.save('npy_dataset/procesed_data.npy', procesed_data)
np.save('npy_dataset/lable_data.npy', lable_data)

#### categorizre the output

In [7]:
from keras.utils import to_categorical
lable_data = to_categorical(lable_data, 10)
lable_data[1]

/home/ahmed/anaconda3/envs/newenvt/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split


from keras import layers
from keras import optimizers

from keras.models import Model
from keras.models import Sequential

from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout

In [11]:
def KerasModel(input_shape):
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation=tf.nn.softmax))

    return model


In [12]:
procesed_data =procesed_data.reshape(2062,100,100,1) #adjust theshape
#split data to test & train
X_train, X_test, Y_train, Y_test = train_test_split(procesed_data, lable_data, test_size=0.3, random_state=101)

model = KerasModel((100,100,1))

model.compile(optimizer = "adam",loss = "binary_crossentropy",metrics = ["accuracy"])



In [13]:
model.fit(x = X_train,y = Y_train,epochs =20 ,batch_size = 16 )

Epoch 1/20
1443/1443 [==============================] - 66s 46ms/step - loss: 0.3585 - acc: 0.8972
Epoch 2/20
1443/1443 [==============================] - 69s 48ms/step - loss: 0.1759 - acc: 0.9307
Epoch 3/20
1443/1443 [==============================] - 62s 43ms/step - loss: 0.1163 - acc: 0.9561
Epoch 4/20
1443/1443 [==============================] - 65s 45ms/step - loss: 0.0847 - acc: 0.9676
Epoch 5/20
1443/1443 [==============================] - 61s 43ms/step - loss: 0.0629 - acc: 0.9753
Epoch 6/20
1443/1443 [==============================] - 60s 42ms/step - loss: 0.0475 - acc: 0.9813
Epoch 7/20
1443/1443 [==============================] - 64s 45ms/step - loss: 0.0344 - acc: 0.9875
Epoch 8/20
1443/1443 [==============================] - 64s 45ms/step - loss: 0.0370 - acc: 0.9851
Epoch 9/20
1443/1443 [==============================] - 61s 42ms/step - loss: 0.0317 - acc: 0.9889
Epoch 10/20
1443/1443 [==============================] - 62s 43ms/step - loss: 0.0320 - acc: 0.9882
Epoch 11/

In [14]:
preds = model.evaluate(x = X_test,y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

619/619 [==============================] - 5s 8ms/step

Loss = 0.14370174718018688
Test Accuracy = 0.9657512262872809
